In [ ]:
import pathlib


class Config_Dataset_Unbinned_Sets:

    def __init__(
        self,
        level,
        split,
        num_events_per_set,
        num_sets_per_label,
        q_squared_veto,
        balanced_classes,
        std_scale,
        shuffle,
        path_dir_dsets_main,
        path_dir_raw_signal,
        label_subset=None,
        is_sensitivity_dataset=False,
        frac_bkg=None,
        path_dir_raw_bkg=None,
    ):
        
        self.level = level
        self.split = split
        self.num_events_per_set = num_events_per_set
        self.num_sets_per_label = num_sets_per_label
        self.q_squared_veto = q_squared_veto
        self.balanced_classes = balanced_classes
        self.std_scale = std_scale
        self.shuffle = shuffle
        self.path_dir_dsets_main = path_dir_dsets_main
        self.path_dir_raw_signal = path_dir_raw_signal
        self.label_subset = label_subset
        self.is_sensitivity_dataset = is_sensitivity_dataset
        self.frac_bkg = frac_bkg
        self.path_dir_raw_bkg = path_dir_raw_bkg


        
        

        

In [1]:
import pathlib

pathlib.Path(None)

TypeError: argument should be a str or an os.PathLike object where __fspath__ returns a str, not 'NoneType'